In [1]:
from eventtrader.keys import SEC_API_KEY

Loading .env from: /Users/faisal/Desktop/Faisal/EventTrader/.env


### Search SEC Filings With Python
https://sec-api.io/docs/query-api/python-example

- find SEC filings by ticker, form type, and publication date

In [2]:

from sec_api import QueryApi

queryApi = QueryApi(api_key=SEC_API_KEY)

search_params = {
    "query": "ticker:TSLA",
    "from": "0",
    "size": "50",
    "sort": [{"filedAt": {"order": "desc"}}],
}

response = queryApi.get_filings(search_params)

response['filings'] gives you a list of all filings

In [23]:
type(response['filings']), len(response['filings'])

(list, 50)

In [5]:
response.keys()

dict_keys(['total', 'query', 'filings'])

In [7]:
response['total']

{'value': 1602, 'relation': 'eq'}

In [9]:
response['query']


{'from': 0, 'size': 50}

In [12]:
len(response['filings'])

50

In [16]:
(response['filings'])[3].keys()

dict_keys(['ticker', 'formType', 'accessionNo', 'cik', 'companyNameLong', 'companyName', 'linkToFilingDetails', 'description', 'linkToTxt', 'filedAt', 'documentFormatFiles', 'periodOfReport', 'entities', 'id', 'seriesAndClassesContractsInformation', 'linkToHtml', 'linkToXbrl', 'dataFiles'])

In [18]:
import json
import pandas as pd

# Load JSON file
file_path = "sec_filings.json"
with open(file_path, "r", encoding="utf-8") as file:
    filings_data = json.load(file)

# Flatten function to extract nested fields
def flatten_filing(filing):
    base_fields = {
        "id": filing.get("id"),
        "accessionNo": filing.get("accessionNo"),
        "cik": filing.get("cik"),
        "ticker": filing.get("ticker"),
        "companyName": filing.get("companyName"),
        "companyNameLong": filing.get("companyNameLong"),
        "formType": filing.get("formType"),
        "description": filing.get("description"),
        "filedAt": filing.get("filedAt"),
        "linkToTxt": filing.get("linkToTxt"),
        "linkToHtml": filing.get("linkToHtml"),
        "linkToXbrl": filing.get("linkToXbrl"),
        "linkToFilingDetails": filing.get("linkToFilingDetails"),
        "periodOfReport": filing.get("periodOfReport"),
        "effectivenessDate": filing.get("effectivenessDate"),
    }

    # Extract Entities
    entities = filing.get("entities", [])
    entity_rows = []
    for entity in entities:
        entity_row = base_fields.copy()
        entity_row.update({
            "entity_companyName": entity.get("companyName"),
            "entity_cik": entity.get("cik"),
            "entity_irsNo": entity.get("irsNo"),
            "entity_stateOfIncorporation": entity.get("stateOfIncorporation"),
            "entity_fiscalYearEnd": entity.get("fiscalYearEnd"),
            "entity_type": entity.get("type"),
            "entity_act": entity.get("act"),
            "entity_fileNo": entity.get("fileNo"),
            "entity_filmNo": entity.get("filmNo"),
            "entity_sic": entity.get("sic"),
        })
        entity_rows.append(entity_row)

    # Extract Documents
    documents = filing.get("documentFormatFiles", [])
    doc_rows = []
    for doc in documents:
        doc_row = base_fields.copy()
        doc_row.update({
            "doc_sequence": doc.get("sequence"),
            "doc_description": doc.get("description"),
            "doc_documentUrl": doc.get("documentUrl"),
            "doc_type": doc.get("type"),
            "doc_size": doc.get("size"),
        })
        doc_rows.append(doc_row)

    # Extract Data Files
    data_files = filing.get("dataFiles", [])
    data_rows = []
    for data in data_files:
        data_row = base_fields.copy()
        data_row.update({
            "data_sequence": data.get("sequence"),
            "data_description": data.get("description"),
            "data_documentUrl": data.get("documentUrl"),
            "data_type": data.get("type"),
            "data_size": data.get("size"),
        })
        data_rows.append(data_row)

    # Extract Series and Classes Contracts Information
    series_classes = filing.get("seriesAndClassesContractsInformation", [])
    series_rows = []
    for series in series_classes:
        series_row = base_fields.copy()
        series_row.update({
            "series_id": series.get("series"),
            "series_name": series.get("name"),
        })
        for contract in series.get("classesContracts", []):
            contract_row = series_row.copy()
            contract_row.update({
                "classContract": contract.get("classContract"),
                "classContract_name": contract.get("name"),
                "classContract_ticker": contract.get("ticker"),
            })
            series_rows.append(contract_row)

    return entity_rows + doc_rows + data_rows + series_rows



In [20]:

# Process all filings
all_filing_rows = []
for filing in response['filings']:
    all_filing_rows.extend(flatten_filing(filing))

# Create DataFrame
df = pd.DataFrame(all_filing_rows)

df.head(2)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,...,doc_sequence,doc_description,doc_documentUrl,doc_type,doc_size,data_sequence,data_description,data_documentUrl,data_type,data_size
0,20a4e6436473402f9d678142b538bbf0,0001494731-25-000001,1318605,TSLA,"Tesla, Inc.","Tesla, Inc. (Issuer)",4,Form 4 - Statement of changes in beneficial ow...,2025-02-10T18:40:33-05:00,https://www.sec.gov/Archives/edgar/data/131860...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20a4e6436473402f9d678142b538bbf0,0001494731-25-000001,1318605,TSLA,"Tesla, Inc.","Tesla, Inc. (Issuer)",4,Form 4 - Statement of changes in beneficial ow...,2025-02-10T18:40:33-05:00,https://www.sec.gov/Archives/edgar/data/131860...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.to_csv("sec_restAPIOutput.csv", index=False)

In [21]:
df.tail(2)

,id,accessionNo,cik,ticker,companyName,companyNameLong,formType,description,filedAt,linkToTxt,...,doc_sequence,doc_description,doc_documentUrl,doc_type,doc_size,data_sequence,data_description,data_documentUrl,data_type,data_size
404,0c594788a057dbb3b45cd22002fb0ab7,0001104659-24-070994,1318605,TSLA,"Tesla, Inc.","Tesla, Inc. (Filer)",DEFA14A,Form DEFA14A - Additional definitive proxy sol...,2024-06-13T06:02:30-04:00,https://www.sec.gov/Archives/edgar/data/131860...,...,6,GRAPHIC,https://www.sec.gov/Archives/edgar/data/131860...,GRAPHIC,23048,NaN,NaN,NaN,NaN,NaN
405,0c594788a057dbb3b45cd22002fb0ab7,0001104659-24-070994,1318605,TSLA,"Tesla, Inc.","Tesla, Inc. (Filer)",DEFA14A,Form DEFA14A - Additional definitive proxy sol...,2024-06-13T06:02:30-04:00,https://www.sec.gov/Archives/edgar/data/131860...,...,,Complete submission text file,https://www.sec.gov/Archives/edgar/data/131860...,,389328,NaN,NaN,NaN,NaN,NaN


In [ ]:
df

In [19]:
filing = (response['filings'])[3]
flatten_filing(filing)

[{'id': 'd483c3503b775a7989306cf2048bcd4b',
  'accessionNo': '0000950170-25-014794',
  'cik': '1318605',
  'ticker': 'TSLA',
  'companyName': 'Tesla, Inc.',
  'companyNameLong': 'Tesla, Inc. (Issuer)',
  'formType': '4',
  'description': 'Form 4 - Statement of changes in beneficial ownership of securities',
  'filedAt': '2025-02-05T19:00:07-05:00',
  'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1318605/000095017025014794/0000950170-25-014794.txt',
  'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1318605/000095017025014794/0000950170-25-014794-index.htm',
  'linkToXbrl': '',
  'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1242782/000095017025014794/xslF345X05/ownership.xml',
  'periodOfReport': '2025-02-03',
  'effectivenessDate': None,
  'entity_companyName': 'DENHOLM ROBYN M (Reporting)',
  'entity_cik': '1242782',
  'entity_irsNo': None,
  'entity_stateOfIncorporation': None,
  'entity_fiscalYearEnd': None,
  'entity_type': '4',
  'entity_act': '34'

In [17]:
(response['filings'])[3]

{'ticker': 'TSLA',
 'formType': '4',
 'accessionNo': '0000950170-25-014794',
 'cik': '1318605',
 'companyNameLong': 'Tesla, Inc. (Issuer)',
 'companyName': 'Tesla, Inc.',
 'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1242782/000095017025014794/xslF345X05/ownership.xml',
 'description': 'Form 4 - Statement of changes in beneficial ownership of securities',
 'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1318605/000095017025014794/0000950170-25-014794.txt',
 'filedAt': '2025-02-05T19:00:07-05:00',
 'documentFormatFiles': [{'sequence': '1',
   'size': '\xa0',
   'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1242782/000095017025014794/xslF345X05/ownership.xml',
   'description': '4',
   'type': '4'},
  {'sequence': '1',
   'size': '34289',
   'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1242782/000095017025014794/ownership.xml',
   'description': '4',
   'type': '4'},
  {'sequence': '\xa0',
   'size': '35754',
   'documentUrl': 'https://www.se

In [3]:
response

{'total': {'value': 1602, 'relation': 'eq'},
 'query': {'from': 0, 'size': 50},
 'filings': [{'ticker': 'TSLA',
   'formType': '4',
   'accessionNo': '0001494731-25-000001',
   'cik': '1318605',
   'companyNameLong': 'Tesla, Inc. (Issuer)',
   'companyName': 'Tesla, Inc.',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1318605/000149473125000001/xslF345X05/edgardoc.xml',
   'description': 'Form 4 - Statement of changes in beneficial ownership of securities',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1318605/000149473125000001/0001494731-25-000001.txt',
   'filedAt': '2025-02-10T18:40:33-05:00',
   'documentFormatFiles': [{'sequence': '1',
     'size': '\xa0',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1318605/000149473125000001/xslF345X05/edgardoc.xml',
     'description': 'PRIMARY DOCUMENT',
     'type': '4'},
    {'sequence': '1',
     'size': '5345',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1318605/000149473125

### Extractor API

https://sec-api.io/docs/sec-filings-item-extraction-api

python : https://sec-api.io/docs/sec-filings-item-extraction-api/python-example

### Access financial statements in HTML format -:
 https://sec-api.io/docs/financial-statements#3-access-financial-statements-in-html-format

- Every 10-K/10-Q provides separate financial statements in different links.                    

    Income Statement (https://www.sec.gov/Archives/edgar/data/<cik>/<LinkccessionNo>/R4.htm) & so on - see link
